<a href="https://colab.research.google.com/github/LorenzoDonatti/T1_confiabilidade_SEP_LorenzoDonatti/blob/main/T1_confiabilidade_SEP_LorenzoDonatti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Primeiro Passo
---



> Adicione um arquivo .pwf com as contingencias de primeira ordem necessárias


> A partir deste arquivo uma lista de listas com todas as contingencias de 1 ordem com os códigos das barras será gerada





In [ ]:
import numpy as np

# Abra o arquivo em modo de leitura
with open('/content/STSB32_Cont_1ordem.pwf', 'rb') as arquivo:
    dados = []
    lendo_contingencia = False

    # Leia o conteúdo do arquivo linha por linha
    for linha in arquivo:
        linha = linha.decode('iso-8859-1').strip()  # Remove espaços em branco no início e no fim da linha
        #print(linha)
        # Verifique se é o início de uma nova contingência
        if linha.startswith('DCTG'):
            lendo_contingencia = True
        elif linha.startswith('99999'):
            lendo_contingencia = False
        elif lendo_contingencia and not (linha.startswith('CIRD') or linha.startswith('FCAS')):
            # Divida a linha em elementos individuais (usando espaço como delimitador)
            elementos = linha.split()
            if elementos and elementos[2]=='1':
                valor1 = int(elementos[5])
                valor2 = int(elementos[7])
                dados.append([valor1,valor2])
# Agora, lista_de_listas conterá os valores das contingências lidos do arquivo
LT = np.array(dados)
print(LT)
print(len(LT))

[[ 839 1047]
 [ 839 2458]
 [ 856  933]
 [ 856 1060]
 [ 896  897]
 [ 933  895]
 [ 933  955]
 [ 933  959]
 [ 934  960]
 [ 934 1047]
 [ 938  946]
 [ 938  955]
 [ 938  959]
 [ 947  939]
 [ 955  946]
 [ 955  964]
 [ 955  979]
 [ 959  895]
 [ 960 1015]
 [ 964  976]
 [ 965 1057]
 [ 976  979]
 [ 995  964]
 [ 995  979]
 [ 995 1030]
 [ 995 1060]
 [ 999  896]
 [ 999  933]
 [ 999 1060]
 [1010  947]
 [1015  939]
 [1030  955]
 [1047 1069]
 [1057 1010]
 [1060  897]
 [1069 1041]
 [1041  963]
 [ 963  965]]
38


# Segundo Passo
---



> Rodar uma análise de contingencias de primeira ordem no ANAREDE e abrir o relatório gerado.

> A partir deste relatório, o número das contingencias severas será obtido e elas serão removidas da lista de contingências.

> Estas contingencias severas serão os cortes de 1 ordem.

In [ ]:
# Abra o arquivo em modo de leitura
with open('/content/relatorio_contingencias_1ordem.txt', 'rb') as arquivo:
    dados = []
    lendo_caso = False
    lendo_div = False

    # Leia o conteúdo do arquivo linha por linha
    for linha in arquivo:
        linha = linha.decode('iso-8859-1').strip()  # Remove espaços em branco no início e no fim da linha
        #print(linha)
        # Verifique se é o início de uma nova contingência
        if linha.startswith('SUMARIO DE MONITORACAO  ( CASOS MAIS SEVEROS )'):
            lendo_caso = True
        elif linha.startswith('DIVERGENTES') or linha.startswith('NAO CONVERGENTES'):
            lendo_caso = True
            lendo_div = True
        elif lendo_caso and linha !='':
            # Divida a linha em elementos individuais (usando espaço como delimitador)
            elementos = linha.split()
            if elementos[0].isdigit():
                dados.append(int(elementos[0]))
                if lendo_div:
                   dados.extend([int(i) for i in elementos[1:]])
# Agora, lista_de_listas conterá os valores das contingências lidos do arquivo
remove_1 = sorted(np.array(dados) - 1)
print(remove_1)

[15, 16, 19, 22, 23]


In [ ]:
cortes_1ordem = LT[remove_1]
cortes_1ordem

array([[955, 964],
       [955, 979],
       [964, 976],
       [995, 964],
       [995, 979]])

In [ ]:
# Remove as linhas da lista inicial (contingências de 1ª ordem)
LT2 = np.delete(LT,remove_1,axis=0)
m=len(LT2)

print(LT2)
print(m)

[[ 839 1047]
 [ 839 2458]
 [ 856  933]
 [ 856 1060]
 [ 896  897]
 [ 933  895]
 [ 933  955]
 [ 933  959]
 [ 934  960]
 [ 934 1047]
 [ 938  946]
 [ 938  955]
 [ 938  959]
 [ 947  939]
 [ 955  946]
 [ 959  895]
 [ 960 1015]
 [ 965 1057]
 [ 976  979]
 [ 995 1030]
 [ 995 1060]
 [ 999  896]
 [ 999  933]
 [ 999 1060]
 [1010  947]
 [1015  939]
 [1030  955]
 [1047 1069]
 [1057 1010]
 [1060  897]
 [1069 1041]
 [1041  963]
 [ 963  965]]
33


# Terceiro Passo
---

> Criar uma lista com todas combinações possíveis entre as contingencias não severas para realizar a analise de segunda ordem.

> Com essas combinações, criar um arquivo .PWF semelhante ao de primeira ordem, porém com todas contingencias de segunda ordem.

In [ ]:
from itertools import combinations

# Cria uma lista de todos as linhas de 1 a m
linhas = list(range(0, m))

# Usa a função numpy.array() para retornar uma matriz de todas as combinações de dois elementos das linhas
combinacoes = np.array(list(combinations(linhas, 2)))
n = len(combinacoes)

print(combinacoes)
print(n)

[[ 0  1]
 [ 0  2]
 [ 0  3]
 ...
 [30 31]
 [30 32]
 [31 32]]
528


In [ ]:
# Abre o arquivo para escrita do PWF
with open('/content/STSB32_Cont_2ordem.pwf', 'w') as fileID:

  # Comando para listar as contingências
  fileID.write('DCTG\n')

  # Número da cont., prioridade e descrição
  for i in range(n):
    fileID.write('%4d 0  2 Contingencia dupla de %d para %d e de %d para %d\n' % (i+200, LT2[combinacoes[i,0], 0], LT2[combinacoes[i,0], 1], LT2[combinacoes[i,1], 0], LT2[combinacoes[i,1], 1]))

    # Linha, aqui tá sempre o circuito 1
    fileID.write('CIRD %4d  %4d   1\n' %  (LT2[combinacoes[i,0], 0], LT2[combinacoes[i,0], 1]))
    fileID.write('CIRD %4d  %4d   1\n' % (LT2[combinacoes[i,1], 0], LT2[combinacoes[i,1], 1]))

    # Fim do Caso
    fileID.write('FCAS\n')

  # FIM do comando DCTG
  fileID.write('99999\n')

  # FIM do arquivo PWF
  fileID.write('FIM\n')

# Quarto Passo
---

> Assim como no segundo passo, realizar a análise de contingencias no ANAREDE, porém, neste caso, de segunda ordem, e abrir o relatório gerado.

> Com base no relatório, obter no número das contingências severas.

> Estas contingencias severas serão os cortes de 2 ordem.

In [ ]:
# Abra o arquivo em modo de leitura
with open('/content/relatorio_contingencias_2ordem.txt', 'rb') as arquivo:
    dados = []
    lendo_caso = False
    lendo_div = False

    # Leia o conteúdo do arquivo linha por linha
    for linha in arquivo:
        linha = linha.decode('iso-8859-1').strip()  # Remove espaços em branco no início e no fim da linha
        #print(linha)
        # Verifique se é o início de uma nova contingência
        if linha.startswith('SUMARIO DE MONITORACAO  ( CASOS MAIS SEVEROS )'):
            lendo_caso = True
        elif linha.startswith('DIVERGENTES') or linha.startswith('NAO CONVERGENTES'):
            lendo_caso = True
            lendo_div = True
        elif lendo_caso and linha !='':
            # Divida a linha em elementos individuais (usando espaço como delimitador)
            elementos = linha.split()
            if elementos[0].isdigit():
                dados.append(int(elementos[0]))
                if lendo_div:
                   dados.extend([int(i) for i in elementos[1:]])
# Agora, lista_de_listas conterá os valores das contingências lidos do arquivo
#remove = sorted(np.array(dados))
remove_2 = np.array(dados)
print(remove_2)

[476 475 559 614 358 558 557 538 521 228 669 633 519 499 497 478 409 406
 338]


In [ ]:
index = combinacoes[np.array(remove_2) - 200]

cortes_2ordem = LT2[combinacoes[np.array(remove_2) - 200]]
print(cortes_2ordem)

[[[ 938  946]
  [ 938  959]]

 [[ 938  946]
  [ 938  955]]

 [[ 955  946]
  [ 965 1057]]

 [[ 965 1057]
  [1010  947]]

 [[ 933  895]
  [ 955  946]]

 [[ 955  946]
  [ 960 1015]]

 [[ 955  946]
  [ 959  895]]

 [[ 947  939]
  [ 955  946]]

 [[ 938  959]
  [ 960 1015]]

 [[ 839 1047]
  [1060  897]]

 [[ 999  896]
  [1060  897]]

 [[ 976  979]
  [1060  897]]

 [[ 938  959]
  [ 955  946]]

 [[ 938  955]
  [ 955  946]]

 [[ 938  955]
  [ 938  959]]

 [[ 938  946]
  [ 955  946]]

 [[ 933  959]
  [ 955  946]]

 [[ 933  959]
  [ 938  955]]

 [[ 896  897]
  [ 999  896]]]


#Quinto Passo
---

> Como na descrição do trabalho, os valores de Q estão associados com os nomes das barras, e não com os códigos, será necessária uma pequena conversão.

> Abrindo o arquivo .PWF base do STSB32 é possível acessar a relação NOME - CÓDIGO da barra.

> Além disso, um dicionário relacionando o valor de Q com as LTs será criado e as listas com os números das barras será substituída pelos valores de Q.

In [ ]:
# Abra o arquivo em modo de leitura
with open('/content/STSB32_EDITADO.PWF', 'rb') as arquivo:
    nomes = {}
    lendo_caso = False

    # Leia o conteúdo do arquivo linha por linha
    for linha in arquivo:
        linha = linha.decode('iso-8859-1').strip()  # Remove espaços em branco no início e no fim da linha
        # Verifique se é o início de uma nova contingência
        if linha.startswith('DBAR'):
            lendo_caso = True
        elif linha.startswith('99999'):
            lendo_caso = False
        elif lendo_caso and not linha.startswith('(Num)'):
            # Divida a linha em elementos individuais (usando espaço como delimitador)
            elementos = linha.split()
            nomes[elementos[2][1:]] = int(elementos[0])
# Agora, lista_de_listas conterá os valores das contingências lidos do arquivo
print(nomes)

{'BATEIAS-230': 814, 'CASCAV-PR230': 839, 'SEGRED-PR525': 856, 'BATEIA-PR525': 895, 'CASCOE-PR525': 896, 'SCAXIA-PR525': 897, 'AREIA--PR525': 933, 'AREIA-230': 934, 'BLUMEN-SC525': 938, 'BLUMEN-SC230': 939, 'BIGUAC-SC525': 946, 'BIGUAC-SC230': 947, 'CNOVOS-SC525': 955, 'CURITI-PR525': 959, 'CURITI-PR230': 960, 'MCLARO-RS230': 963, 'CAXIAS-RS525': 964, 'CAXIAS-RS230': 965, 'GRAVAT-RS525': 976, 'NSRITA-RS525': 979, 'ITA-----0425': 995, 'IVAIPE-PR525': 999, 'JLAC-B-SC230': 1010, 'JOINOR-SC230': 1015, 'MACHAD-SC525': 1030, 'PFUNDO-RS230': 1041, 'SOSORIO-230': 1047, 'SIDERO-SC230': 1057, 'SSANTI-PR525': 1060, 'XANXER-SC230': 1069, 'GRAVA2-RS230': 1210, 'CASCOE-PR230': 2458}


In [ ]:
import pandas as pd
from io import StringIO

data = '''AREIA--PR525 SEGRED-PR525 2,7883E-04
ITA-----0425 MACHAD-SC525 3,1767E-04
AREIA--PR525 BATEIA-PR525 1,2577E-03
ITA-----0425 SSANTI-PR525 9,1902E-04
AREIA--PR525 CNOVOS-SC525 8,6648E-04
IVAIPE-PR525 SSANTI-PR525 8,2081E-04
AREIA--PR525 CURITI-PR525 1,1556E-03
SCAXIA-PR525 SSANTI-PR525 4,4252E-04
AREIA--PR525 IVAIPE-PR525 8,5126E-04
SEGRED-PR525 SSANTI-PR525 2,9751E-04
BATEIA-PR525 CURITI-PR525 1,6476E-04
AREIA-230 CURITI-PR230 6,5850E-04
BIGUAC-SC525 BLUMEN-SC525 4,2778E-04
AREIA-230 SOSORIO-230 4,2973E-04
BIGUAC-SC525 CNOVOS-SC525 1,1535E-03
BIGUAC-SC230 BLUMEN-SC230 3,3626E-04
BLUMEN-SC525 CNOVOS-SC525 1,2405E-03
BIGUAC-SC230 JLAC-B-SC230 3,4676E-04
BLUMEN-SC525 CURITI-PR525 6,7837E-04
BLUMEN-SC230 JOINOR-SC230 1,9550E-04
CASCOE-PR525 SCAXIA-PR525 3,0981E-04
CASCAV-PR230 SOSORIO-230 2,1504E-04
CASCOE-PR525 IVAIPE-PR525 1,0270E-03
CASCAV-PR230 CASCOE-PR230 2,7589E-05
CAXIAS-RS525 CNOVOS-SC525 9,9905E-04
CAXIAS-RS230 MCLARO-RS230 1,4267E-04
CAXIAS-RS525 GRAVAT-RS525 3,8747E-04
CAXIAS-RS230 SIDERO-SC230 5,6385E-04
CAXIAS-RS525 ITA-----0425 1,2528E-03
CURITI-PR230 JOINOR-SC230 2,6752E-04
CNOVOS-SC525 NSRITA-RS525 1,2646E-03
JLAC-B-SC230 SIDERO-SC230 1,2668E-04
CNOVOS-SC525 MACHAD-SC525 1,9623E-04
MCLARO-RS230 PFUNDO-RS230 6,1219E-04
GRAVAT-RS525 NSRITA-RS525 1,4509E-04
PFUNDO-RS230 XANXER-SC230 2,1210E-04
ITA-----0425 NSRITA-RS525 1,5422E-03
SOSORIO-230 XANXER-SC230 4,3374E-04'''

df = pd.read_csv(StringIO(data), sep=' ', names=['Origem', 'Destino', 'Valor'])

df['Valor'] = pd.to_numeric(df['Valor'].str.replace(',','.'))

# Crie um dicionário para armazenar os valores de confiabilidade mapeados por pares [origem, destino]
pfalha = {}

for _, row in df.iterrows():
    origem = nomes.get(row['Origem'], row['Origem'])  # Use o nome original se não estiver no dicionário
    destino = nomes.get(row['Destino'], row['Destino'])  # Use o nome original se não estiver no dicionário
    valor = row['Valor']

    par_1 = [origem, destino]
    par_2 = [destino, origem]

    pfalha[tuple(par_1)] = valor
    pfalha[tuple(par_2)] = valor

# Imprima o dicionário resultante
print(pfalha)

{(933, 856): 0.00027883, (856, 933): 0.00027883, (995, 1030): 0.00031767, (1030, 995): 0.00031767, (933, 895): 0.0012577, (895, 933): 0.0012577, (995, 1060): 0.00091902, (1060, 995): 0.00091902, (933, 955): 0.00086648, (955, 933): 0.00086648, (999, 1060): 0.00082081, (1060, 999): 0.00082081, (933, 959): 0.0011556, (959, 933): 0.0011556, (897, 1060): 0.00044252, (1060, 897): 0.00044252, (933, 999): 0.00085126, (999, 933): 0.00085126, (856, 1060): 0.00029751, (1060, 856): 0.00029751, (895, 959): 0.00016476, (959, 895): 0.00016476, (934, 960): 0.0006585, (960, 934): 0.0006585, (946, 938): 0.00042778, (938, 946): 0.00042778, (934, 1047): 0.00042973, (1047, 934): 0.00042973, (946, 955): 0.0011535, (955, 946): 0.0011535, (947, 939): 0.00033626, (939, 947): 0.00033626, (938, 955): 0.0012405, (955, 938): 0.0012405, (947, 1010): 0.00034676, (1010, 947): 0.00034676, (938, 959): 0.00067837, (959, 938): 0.00067837, (939, 1015): 0.0001955, (1015, 939): 0.0001955, (896, 897): 0.00030981, (897, 896):

#SEXTO PASSO
---
>Percorre as listas e calcula a confiabilidade do sistema.

In [ ]:
valores_q1ordem = []
valores_q2ordem = []

for caminho in cortes_1ordem:
    if tuple(caminho) in pfalha:
        valores_q1ordem.append(pfalha[tuple(caminho)])

for sublista in cortes_2ordem:
    resultado = 1.0  # Inicializa o resultado com 1.0
    for caminho in sublista:
        if tuple(caminho) in pfalha:
            resultado *= pfalha[tuple(caminho)]  # Multiplica o resultado pelo valor correspondente
    valores_q2ordem.append(resultado)

print(valores_q1ordem)
print(valores_q2ordem)

[0.00099905, 0.0012646, 0.00038747, 0.0012528, 0.0015422]
[2.901931186e-07, 5.3066109e-07, 6.50400975e-07, 1.95520626e-07, 1.45075695e-06, 3.0858432e-07, 1.9005066e-07, 3.8787591e-07, 1.814775424e-07, 9.51595008e-08, 4.5446803999999995e-07, 6.420522680000001e-08, 7.82499795e-07, 1.4309167500000001e-06, 8.41517985e-07, 4.9344423e-07, 1.3329845999999998e-06, 1.4335218e-06, 3.1817487e-07]


In [ ]:
valores_q = np.array(valores_q1ordem + valores_q2ordem)
confiabilidade = np.prod(1-valores_q)

print("A confiabilidade do sistema é:")
print(confiabilidade)

0.9945539806314897
